<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_6_tuning_healthnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0   1221      0 --:--:-- --:--:-- --:--:--  1216
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-6.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-6.validation.csv",header=None)
valid.head()

,0,1
0,2,heel pain
1,2,heels hurt
2,4,black and blue marks
3,5,fast and slow heartbeat
4,5,increased heart rate


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-6.test.csv",header=None)
test.head()

,0,1
0,2,left heel pain
1,5,tachycardia
2,8,ARTHRITIS
3,8,arthrites
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_healthnews_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.947228,5.840823,0.141629,02:02


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.778275,5.311156,0.179185,02:01


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.398482,5.257406,0.183705,02:01


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.355101,5.199203,0.192634,02:00


In [0]:
learn.save_encoder('healthnews_fold6_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('healthnews_fold6_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.676902,4.369252,0.310160,01:07


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.978127,4.056021,0.340909,01:23


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.531368,3.699362,0.367647,03:28


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.346044,3.491710,0.405080,03:20
1,3.134640,3.378397,0.417112,03:30


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.991639,3.317201,0.427807,03:19
1,2.954272,3.244614,0.427807,03:09


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.795531,3.175367,0.442513,03:16
1,2.719462,3.107069,0.445187,03:34


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.640953,3.112197,0.439840,03:11
1,2.607423,3.090499,0.450535,03:10


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.604877,3.106846,0.454545,03:12
1,2.519147,3.028065,0.461230,03:17


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.448897,3.031156,0.463904,03:29
1,2.553725,3.085392,0.465241,03:30
2,2.361994,2.959875,0.467914,03:23
3,2.387719,3.028576,0.466578,03:16


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.281605,3.026178,0.474599,03:26
1,2.439851,3.020167,0.471925,03:12
2,2.274751,2.944752,0.478610,03:36
3,2.206790,2.958007,0.483957,03:35


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.143714,2.978162,0.470588,03:25
1,2.266320,3.066355,0.475936,03:24
2,2.123514,2.955612,0.491979,03:19
3,2.124037,3.021659,0.487968,03:39


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.253182,3.085272,0.481283,03:22
1,2.245786,3.134672,0.475936,03:22
2,2.173381,3.059519,0.483957,03:11
3,2.118849,3.061593,0.482620,03:30


In [0]:
# save the best model

learn.save_encoder('ask-6-tuning-healthnews')

In [42]:
learn.load_encoder('ask-6-tuning-healthnews')
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.034252,3.035380,0.485294,01:11
1,2.156398,3.120201,0.479947,01:10
2,2.138639,3.108872,0.485294,01:05
3,2.044392,3.040918,0.486631,01:05


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.102480,3.037245,0.485294,01:09
1,2.155690,3.077594,0.477273,01:10
2,2.099928,3.117231,0.481283,01:10
3,2.097683,3.048601,0.478610,01:08


In [44]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.081723,3.142864,0.478610,01:07
1,2.128500,3.136709,0.475936,01:07
2,2.174222,3.151323,0.471925,01:07
3,2.040095,3.045460,0.477273,01:10


In [45]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.129014,3.148642,0.471925,01:06
1,2.169361,3.169306,0.482620,01:10
2,2.131218,3.053360,0.483957,01:11
3,2.025200,3.115994,0.481283,01:10


In [46]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.064873,3.165763,0.477273,01:11
1,2.091671,3.066109,0.475936,01:10
2,2.174752,3.250118,0.481283,01:06
3,2.104237,3.143055,0.481283,01:09


In [47]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.117715,3.219455,0.478610,01:07
1,2.063009,3.089059,0.491979,01:09
2,2.174305,3.161307,0.486631,01:09
3,1.996634,3.102412,0.477273,01:12


# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 1026, 8, 1026, 8, 8, 8, 8, 8, 8, 8, 339, 227, 27, 584, 1035, 639, 37, 37, 37, 37, 37, 289, 775, 639, 639, 402, 1002, 326, 814, 409, 49, 1026, 326, 711, 711, 711, 711, 711, 711, 790, 326, 711, 711, 711, 711, 326, 321, 265, 265, 577, 343, 135, 326, 877, 877, 289, 864, 210, 128, 228, 711, 711, 711, 265, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 289, 172, 711, 711, 711, 711, 80, 80, 80, 80, 80, 80, 175, 491, 711, 210, 711, 326, 711, 711, 711, 711, 326, 49, 711, 711, 711, 711, 711, 711, 711, 228, 96, 711, 711, 214, 546, 214, 214, 214, 228, 265, 33, 108, 108, 108, 108, 788, 224, 111, 805, 877, 326, 1035, 711, 315, 775, 117, 289, 711, 864, 711, 817, 125, 711, 125, 125, 125, 125, 125, 125, 125, 127, 910, 319, 135, 443, 402, 132, 151, 151, 711, 52, 52, 711, 165, 117, 309, 864, 864, 864, 1035, 711, 825, 174, 182, 182, 182, 183, 183, 183, 183, 711, 711, 240, 240, 187, 189, 190, 190, 1035, 289, 197,

In [50]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

214
0.2471131639722864
